In [1]:
%matplotlib inline

import contextlib
import cooler
import h5py
import math
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import cm
from PIL import Image

In [2]:
cooler_file_rao = '/Users/Fritz/Code/4DN/higlass-server/media/hg19/Rao2014-GM12878-MboI-allreps-filtered.1kb.multires.cool'

# Chrom 22
locus1 = 2829728720
locus2 = 2829728720 + 51304566
dim_1 = int(math.ceil(51304566.0 / 16000))

# Chrom 15
locus3 = 2307285719
locus4 = 2307285719 + 102531392
dim_2 = int(math.ceil(102531392.0 / 16000))

chr9_1 = 0
chr9_2 = 1

chrom = 1
loop_list='/Volumes/FFRITZ/4DN/data/rao/GSE63525_GM12878_primary+replicate_HiCCUPS_looplist.txt'
domain_list='/Volumes/FFRITZ/4DN/data/rao/GSE63525_GM12878_primary+replicate_Arrowhead_domainlist.txt'

In [3]:
import cooler
import h5py
import logging
import numpy as np
import pandas as pd

logger = logging.getLogger(__name__)


# Methods

def get_chrom_names_cumul_len(c):
    '''
    Get the chromosome names and cumulative lengths:

    Args:

    c (Cooler): A cooler file

    Return:

    (names, sizes, lengths) -> (list(string), dict, np.array(int))
    '''

    chrom_sizes = {}
    chrom_cum_lengths = [0]
    chrom_ids = {}
    chroms = []

    k = 0

    for chrom in c.chroms():
        (name, length) = chrom.as_matrix()[0]

        chroms += [name]
        chrom_cum_lengths += [chrom_cum_lengths[-1] + length]
        chrom_sizes[name] = length
        chrom_ids[name] = k

        k += 1

    return (chroms, chrom_sizes, np.array(chrom_cum_lengths), chrom_ids)


def get_intra_chr_loops_from_looplist(loop_list, chr=0):
    loops = pd.DataFrame.from_csv(loop_list, sep='\t', header=0)

    s_chr = str(chr)

    if chr > 0:
        loops = loops[loops['chr2'] == s_chr].loc[s_chr]

    chrs = np.zeros((loops.shape[0], 2), dtype=object)

    chrs[:, 0] = loops['chr2']
    chrs[:, 1] = loops['chr2']

    return (loops.as_matrix()[:, [0, 1, 3, 4]], chrs)


def rel_2_abs_loci(loci, chr_info):
    '''
    chr_info[0] = chromosome names
    chr_info[1] = chromosome lengths
    chr_info[2] = cumulative lengths
    chr_info[3] = chromosome ids
    '''
    def absolutize(chr, x, y):
        offset = chr_info[2][chr_info[3]['chr%s' % str(chr)]]

        return (offset + x, offset + y)

    def absolutize_tuple(tuple):
        return (
            absolutize(*tuple[0:3]) +
            absolutize(*tuple[3:6])
        )

    return map(absolutize_tuple, loci)


def get_cooler(f, zoomout_level=0):
    c = None

    try:
        zoom_levels = np.array(f.keys(), dtype=int)

        max_zoom = np.max(zoom_levels)
        min_zoom = np.min(zoom_levels)

        zoom_level = max_zoom - max(zoomout_level, 0)

        if (zoom_level >= min_zoom and zoom_level <= max_zoom):
            c = cooler.Cooler(f[str(zoom_level)])
        else:
            c = cooler.Cooler(f['0'])

        return c
    except Exception as e:
        logger.error(e)
        pass  # failed loading zoomlevel of cooler file

    try:
        c = cooler.Cooler(f)
    except Exception:
        logger.error(e)

    return c


def get_frag_by_loc(
    cooler_file,
    loci,
    is_rel=True,
    dim=22,
    balanced=True,
    zoomout_level=0
):
    with h5py.File(cooler_file, 'r') as f:
        c = get_cooler(f, zoomout_level)

        fragments = collect_frags(c, loci, is_rel, dim, balanced)

    return fragments


def calc_measure_dtd(matrix, locus):
    '''
    Calculate the distance to the diagonal
    '''
    return np.abs(locus['end1'] - locus['start2'])


def calc_measure_size(matrix, locus, bin_size=1):
    '''
    Calculate the size of the snippet
    '''
    return (
        np.abs(locus['start1'] - locus['end1']) *
        np.abs(locus['start2'] - locus['end2'])
    ) / bin_size


def calc_measure_noise(matrix):
    '''
    Estimate the noise level of the input matrix using the standard deviation
    '''
    low_quality_bins = np.where(matrix == -1)

    # Assign 0 for now to avoid influencing the standard deviation
    matrix[low_quality_bins] = 0

    noise = np.std(matrix)

    # Ressign -1 to low quality bins
    matrix[low_quality_bins] = -1

    return noise


def calc_measure_sharpness(matrix):
    low_quality_bins = np.where(matrix == -1)

    # Assign 0 for now to avoid influencing the variance caluclation
    matrix[low_quality_bins] = 0

    sum = np.sum(matrix)
    sum = sum if sum > 0 else 1
    dim = matrix.shape[0]

    middle = (dim - 1) / 2
    m = dim

    if dim % 2 == 0:
        middle = (dim - 2) / 2
        m = dim / 2

    var = 0

    for i in range(dim):
        for j in range(dim):

            var += (
                ((i - (middle + i // m)) ** 2 + (j - middle + i // m) ** 2) *
                matrix[i, j]
            )

    # Ressign -1 to low quality bins
    matrix[low_quality_bins] = -1

    return var / sum


def get_bin_size(cooler_file, zoomout_level=-1):
    with h5py.File(cooler_file, 'r') as f:
        c = get_cooler(f, zoomout_level)

        return c.util.get_binsize()


def collect_frags(c, loci, is_rel=False, dim=22, balanced=True):
    chr_info = get_chrom_names_cumul_len(c)

    if is_rel:
        loci = rel_2_abs_loci(loci, chr_info)

    fragments = np.zeros((len(loci), dim, dim))

    k = 0
    for locus in loci:
        fragments[k] = get_frag(
            c, chr_info, *locus, balanced=balanced, dim=dim
        )

        if max > 0 and k >= max:
            break

        k += 1

    return fragments


def get_chrom(abs_pos, chr_info=None, c=None):
    if chr_info is None:
        try:
            chr_info = get_chrom_names_cumul_len(c)
        except:
            return None

    try:
        chr_id = np.flatnonzero(chr_info[2] > abs_pos)[0] - 1
    except IndexError:
        return None

    return chr_info[0][chr_id]


def get_chroms(abs_pos, chr_info=None, cooler_file=None, zoomout_level=-1):
    chroms = np.zeros((abs_pos.shape[0], 2), dtype=object)

    if chr_info is None:
        with h5py.File(cooler_file, 'r') as f:
            c = get_cooler(f, zoomout_level)
            chr_info = get_chrom_names_cumul_len(c)

    i = 0
    for pos in abs_pos:
        chroms[i] = get_chrom(pos, chr_info)
        i += 1

    return chroms


def rel_loci_2_obj(loci_rel_chroms):
    loci = []

    i = 0
    for locus in loci_rel_chroms:
        loci.append({
            'chrom1': loci_rel_chroms[i, 0],
            'start1': loci_rel_chroms[i, 1],
            'end1': loci_rel_chroms[i, 2],
            'strand1': (
                'coding' if loci_rel_chroms[i, 1] < loci_rel_chroms[i, 2] else
                'noncoding'
            ),
            'chrom2': loci_rel_chroms[i, 3],
            'start2': loci_rel_chroms[i, 4],
            'end2': loci_rel_chroms[i, 5],
            'strand2': (
                'coding' if loci_rel_chroms[i, 1] < loci_rel_chroms[i, 2] else
                'noncoding'
            )
        })
        i += 1

    return loci


def abs_coord_2_bin(c, pos, chr_info):
    try:
        chr_id = np.flatnonzero(chr_info[2] > pos)[0] - 1
    except IndexError:
        return c.info['nbins']

    chrom = chr_info[0][chr_id]
    relPos = pos - chr_info[2][chr_id]

    return c.offset((chrom, relPos, chr_info[1][chrom]))


def get_frag(
    c,
    chr_info,
    start_pos_1,
    end_pos_1,
    start_pos_2,
    end_pos_2,
    padding=10,
    normalize=True,
    balanced=True,
    dim=22
):
    # abs_coord_2_bin(...) returns the inclusive bin ID but in the python world
    # the end position is always exclusive
    start_bin_1 = max(abs_coord_2_bin(c, start_pos_1, chr_info) - padding, 0)
    start_bin_2 = max(abs_coord_2_bin(c, start_pos_2, chr_info) - padding, 0)
    end_bin_1 = abs_coord_2_bin(c, end_pos_1, chr_info) + padding
    end_bin_2 = abs_coord_2_bin(c, end_pos_2, chr_info) + padding

    real_dim = abs(start_bin_1 - end_bin_1)
    if real_dim < dim:
        end_bin_1 += dim - real_dim

    real_dim = abs(start_bin_2 - end_bin_2)
    if real_dim < dim:
        end_bin_2 += dim - real_dim

    pixels = c.matrix(
        as_pixels=True, max_chunk=np.inf, balance=balanced
    )[start_bin_1:end_bin_1, start_bin_2:end_bin_2]

    pixels['index'] = (pixels['bin1_id'] - start_bin_1) * dim + pixels['bin2_id'] - start_bin_2
    
    flat_dim = dim**2

    out = np.zeros(flat_dim, dtype=np.float)

    accessor = 'count'

    if balanced:
        accessor = 'balanced'
    
    out[pixels['index'].values[:flat_dim]] = pixels[accessor].values[:flat_dim]
    
    #for index, row in pixels.iterrows():
    #    try:
    #        out[pixels['index'].values] = pixels[accessor]
    #    except IndexError:
    #        continue

    # Store low quality bins
    low_quality_bins = np.where(np.isnan(out))

    # Assign 0 for now to avoid influencing the max values
    out[low_quality_bins] = 0

    max_val = np.max(out)
    if normalize and max_val > 0:
        out = out / max_val

    # Reassign a special value to cells with low quality
    out[low_quality_bins] = -1

    return out.reshape(dim, dim)[:dim, :dim]


In [4]:
import time

def abs_coord_2_bin(c, pos, chr_info):
    try:
        chr_id = np.flatnonzero(chr_info[2] > pos)[0] - 1
    except IndexError:
        return c.info['nbins']

    chrom = chr_info[0][chr_id]
    relPos = pos - chr_info[2][chr_id]

    return c.offset((chrom, relPos, chr_info[1][chrom]))


def get_cooler(f, zoomout_level=0):
    c = None
    
    try:
        zoom_levels = np.array(f.keys(), dtype=int)

        max_zoom = np.max(zoom_levels)
        min_zoom = np.min(zoom_levels)

        zoom_level = max_zoom - max(zoomout_level, 0)

        if (zoom_level >= min_zoom and zoom_level <= max_zoom):
            c = cooler.Cooler(f[str(zoom_level)])
        else:
            c = cooler.Cooler(f['0'])
    except Exception as e:
        c = cooler.Cooler(f)

    return c


def get_chrom_names_cumul_len(c):
    '''
    Get the chromosome names and cumulative lengths:

    Args:

    c (Cooler): A cooler file

    Return:

    (names, sizes, lengths) -> (list(string), dict, np.array(int))
    '''

    chrom_sizes = {}
    chrom_cum_lengths = [0]
    chrom_ids = {}
    chroms = []

    k = 0

    for chrom in c.chroms():
        (name, length) = chrom.as_matrix()[0]

        chroms += [name]
        chrom_cum_lengths += [chrom_cum_lengths[-1] + length]
        chrom_sizes[name] = length
        chrom_ids[name] = k

        k += 1

    return (chroms, chrom_sizes, np.array(chrom_cum_lengths), chrom_ids)


def get_patch(
    pixels,
    start_bin_1,
    end_bin_1,
    start_bin_2,
    end_bin_2,
    dim,
    normalize=True,
    balanced=True
):
    patch = pixels[start_bin_1:end_bin_1, start_bin_2:end_bin_2]
    patch['index'] = (patch['bin1_id'] - start_bin_1) * dim + patch['bin2_id'] - start_bin_2
    
    out = np.zeros(dim**2, dtype=np.float)

    accessor = 'count'

    if balanced:
        accessor = 'balanced'
        
    out[patch['index'].values] = patch[accessor].values

    return out.reshape(dim, dim)[:dim, :dim]


def get_intra_chr_loops_from_looplist(loop_list, chr=0):
    loops = pd.DataFrame.from_csv(loop_list, sep='\t', header=0)

    s_chr = str(chr)

    if chr > 0:
        loops = loops[loops['chr2'] == s_chr].loc[s_chr]

    chrs = np.zeros((loops.shape[0], 2), dtype=object)

    chrs[:, 0] = loops['chr2']
    chrs[:, 1] = loops['chr2']

    return (loops.as_matrix()[:, [0, 1, 3, 4]], chrs)


def get_loop_loci_by_chr(chrom, loop_list):

    # Get relative loci
    (loci_rel, chroms) = get_intra_chr_loops_from_looplist(
        os.path.join('data', loop_list), chrom
    )

    loci_rel_chroms = np.column_stack(
        (chroms[:, 0], loci_rel[:, 0:2], chroms[:, 1], loci_rel[:, 2:4])
    )

    return loci_rel_chroms


def get_loops_by_chr(cooler_file, chrom, loop_list, zoomout_level=0, dest='rao-gm12878-chr1-loops'):
    loci = get_loop_loci_by_chr(chrom, loop_list)

    num_loops = loci.shape[0]
    batch_size = 100
    
    if not os.path.exists(dest):
        os.makedirs(dest)
        
    start_total = time.clock()
    for b in range(int(math.ceil(num_loops / float(batch_size)))):
        bb = b * batch_size
        
        lloci = loci[bb:(b + 1) * batch_size]
        
        start = time.clock()
        loops = get_frag_by_loc(cooler_file, lloci, True, 22, True, zoomout_level)
        
        print 'Save %d-%d loops in %0.2f sec' % (bb, (b + 1) * batch_size, time.clock() - start)
        
        for i in range(loops.shape[0]):
            plt.imsave(
                '{}/loop-{:05d}.png'.format(dest, bb + i),
                loops[i],
                cmap='gray_r'
            )
        
    print 'Save all loops in %0.2f sec' % (time.clock() - start_total)


def get_patches(cooler_file, locus1, locus2, dim=256, zoomout_base_level=3, base_res=1000, only_base_zoom=False, balance=True, dest='rao-gm12878-chr1', log=False, full=False, scaleUp=1):
    if not os.path.exists(dest):
        os.makedirs(dest)
    
    # Use low level HDF5 API to increase the chunk cache by 5x
    propfaid = h5py.h5p.create(h5py.h5p.FILE_ACCESS)
    settings = list(propfaid.get_cache())
    settings[2] *= 10
    propfaid.set_cache(*settings)

    #with h5py.File(cooler_file, 'r') as f:
    
    with contextlib.closing(h5py.h5f.open(cooler_file, fapl=propfaid)) as fid:
        f = h5py.File(fid)
        

        max_zoom_level = np.max(np.array(f.keys(), dtype=int))
        num_zoomlevels = max_zoom_level - zoomout_base_level
        
        zoomlevels = [max_zoom_level - zoomout_base_level] if only_base_zoom else range(0, num_zoomlevels + 1)
        
        i = 0
        for zoomlevel in zoomlevels:
            zoomout_level = max_zoom_level - zoomlevel
            
            print locus1, locus2
            
            if i > 0:
                break
            
            c = get_cooler(f, zoomout_level)
        
            chr_info = get_chrom_names_cumul_len(c)
            
            start_bin = max(abs_coord_2_bin(c, locus1, chr_info), 0)
            end_bin = abs_coord_2_bin(c, locus2, chr_info)
            num_bins = abs(end_bin - start_bin)
            num_patches = int(math.ceil(num_bins / float(dim)))
            
            print zoomout_level, zoomlevel, num_bins, dim, num_patches**2
            
            pixels = c.matrix(as_pixels=True, max_chunk=np.inf, balance=balance)
            
            real_dim = num_bins if num_bins < dim else dim
            
            print real_dim
            
            avg_time_spent = np.zeros(num_patches, np.float)
            
            for i in range(num_patches):
                start = time.clock()
                for j in range(num_patches):
                    start1 = i * real_dim
                    end1 = (i + 1) * real_dim
                    start2 = j * real_dim
                    end2 = (j + 1) * real_dim

                    patch = get_patch(pixels, start1, end1, start2, end2, dim, False, balance)
                    
                    if log:
                        patch = np.log10(patch)
                    
                    if full:
                        i_lower = np.tril_indices(real_dim, -1)
                        patch[i_lower] = patch.T[i_lower]
                    
                    scaleUp = np.uint8(scaleUp)
                    if scaleUp > 1:
                        patch = np.kron(patch, np.ones((scaleUp,scaleUp)))
                    
                    plt.imsave(
                        '{}/{:05d}-{:05d}-{:05d}-{}-matplotlib.png'.format(dest, zoomlevel, i, j, scaleUp),
                        patch,
                        cmap='gray_r'
                    )
                
                time_spent = time.clock() - start
                avg_time_spent[i] = time_spent
                
                print 'On zoomlevel %d, extracting %d took %0.2f (avg: %0.2f)' % (zoomlevel, num_patches, time_spent, np.sum(avg_time_spent) / (i + 1))
            
            i += 1

In [5]:
get_patches(
    cooler_file=cooler_file_rao,
    locus1=locus3,
    locus2=locus4,
    dim=dim_2,
    zoomout_base_level=4,
    base_res=1000,
    only_base_zoom=True,
    balance=True,
    dest='rao-gm12878-chrM2',
    log=True,
    full=True,
    scaleUp=3
)

2307285719 2409817111
4 10

/Users/Fritz/Virtualenvs/hgs/lib/python2.7/site-packages/ipykernel/__main__.py:216: RuntimeWarning: divide by zero encountered in log10


 6409 6409 1
Let's go 1
6409
On zoomlevel 10, extracting 1 took 143.43 (avg: 143.43)


In [83]:
arr = (np.eye(200)*255).astype('uint8') # sample array
print arr
im = Image.fromarray(arr).convert('RGB') # monochromatic image
# imrgb = Image.merge('RGB', (im,im,im)) # color image
imrgb.show()

[[255   0   0 ...,   0   0   0]
 [  0 255   0 ...,   0   0   0]
 [  0   0 255 ...,   0   0   0]
 ..., 
 [  0   0   0 ..., 255   0   0]
 [  0   0   0 ...,   0 255   0]
 [  0   0   0 ...,   0   0 255]]


In [108]:
get_loops_by_chr(
    cooler_file=cooler_file_rao,
    chrom=1,
    loop_list=loop_list,
    zoomout_level=3,
    balance=False,
    dest='rao-gm12878-chr1-loops-new'
)

Save 0-100 loops in 2.04 sec
Save 100-200 loops in 2.09 sec
Save 200-300 loops in 2.03 sec
Save 300-400 loops in 2.00 sec
Save 400-500 loops in 2.01 sec
Save 500-600 loops in 2.00 sec
Save 600-700 loops in 2.16 sec
Save 700-800 loops in 2.14 sec
Save 800-900 loops in 2.00 sec
Save 900-1000 loops in 0.67 sec
Save all loops in 20.72 sec


In [6]:
# Use low level HDF5 API to increase the chunk cache by 5x
propfaid = h5py.h5p.create(h5py.h5p.FILE_ACCESS)
settings = list(propfaid.get_cache())
settings[2] *= 2
propfaid.set_cache(*settings)

print settings

#with h5py.File(cooler_file, 'r') as f:
    
with contextlib.closing(h5py.h5f.open(cooler_file_rao, fapl=propfaid)) as fid:
    f = h5py.File(fid)
    print(f.id.get_access_plist().get_cache())
    
    c = get_cooler(f, 3)
    pixels = c.matrix(as_pixels=True, max_chunk=np.inf, balance=False)
    
    start = time.clock()
    wurst = pixels[0:2048, 0:2048]
    print '2048x2048 took %0.4f sec' % (time.clock() - start)

[0, 521, 2097152, 0.75]
(0, 521, 2097152, 0.75)
2048x2048 took 0.8780 sec
